In [1]:
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
total_words = 10000
max_review_len = 80
embedding_len = 100

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)

# 文本预处理
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)


batch_size = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(10000).batch(batch_size, 
                                         drop_remainder=True # 如果最后一批次不够128 直接弃掉
                                        )

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batch_size, drop_remainder=True)


print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test shape: (25000, 80)


In [4]:

class MyRNN(keras.Model):
    def __init__(self, units):
        super(MyRNN, self).__init__()
        
        # [b, 80] => [b, 80, 100] 每个句子最多80个单词, 每个单词用100维的向量表示
        # 嵌入模型把你文本转化为数值型的向量
        self.embedding = keras.layers.Embedding(total_words, # 输入维度 目前设置的80维
                                                embedding_len, # 每个单词
                                                input_length=max_review_len, # 句子长度
                                               )
        # cell把你一个句子在时间轴上展开 h_dim: 64 
        # [b, 80, 100] => [b, 64] 
        # # units参数就是h_dim, 将100维的单词向量进行信息提取,转化为内部的state, 就是label的状态
        self.rnn = keras.Sequential([
            # return_sequences 当前层输出的state是否作为下一层的输入; unroll 是否加速运行
            keras.layers.SimpleRNN(units, dropout=0.5, return_sequences=True, unroll=True),
            keras.layers.SimpleRNN(units, dropout=0.5, unroll=True)
        ])
        # 从开始到输出层 数据维度的变化 [b, 80, 100] => [b, 64] => [b, 1]
        # 总共三层 第一层是嵌入层  第二层是SimpleRNNCell 信息提取层 第三层是输出层 输出预测值
        self.out_layer = keras.layers.Dense(1)
        
    def call(self, inputs, training=None):
        """
        前向传播的计算过程
        
        net(x) net(x, training=True) :train mode 
        net(x, training=False): test
        dropout 只有在训练模式下起作用 测试模式下不管用
        :param inputs: 一句话 [b, 80] b为批次大小, 80 每个句子80个单词
        :param training:
        :return:
        """
        x = inputs
        # 嵌入层 [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        
        # rnn cell计算
        x = self.rnn(x)
        
        # 输出层 [b, 64] => [b, 1]
        x = self.out_layer(x)
        # 将这个值 转化为概率 P(y is positive | x)
        prob = tf.sigmoid(x)
        return prob
        
            


In [5]:
h_dim = 64
epochs = 6

# 建立模型
model = MyRNN(h_dim)

# 编译 训练
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(), 
              metrics=['accuracy'])

model.fit(db_train, epochs=epochs, validation_data=db_test)

Epoch 1/6


W0830 15:55:26.199904 4657272256 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


195/195 [==============================] - 29s 151ms/step - loss: 0.5926 - accuracy: 0.5724 - val_loss: 0.4428 - val_accuracy: 0.8074
Epoch 2/6
195/195 [==============================] - 13s 67ms/step - loss: 0.3769 - accuracy: 0.8252 - val_loss: 0.3950 - val_accuracy: 0.8325
Epoch 3/6
195/195 [==============================] - 13s 64ms/step - loss: 0.3239 - accuracy: 0.8664 - val_loss: 0.4212 - val_accuracy: 0.8318
Epoch 4/6
195/195 [==============================] - 13s 64ms/step - loss: 0.2618 - accuracy: 0.8951 - val_loss: 0.4571 - val_accuracy: 0.8274
Epoch 5/6
195/195 [==============================] - 15s 76ms/step - loss: 0.2182 - accuracy: 0.9138 - val_loss: 0.5320 - val_accuracy: 0.7907
Epoch 6/6
195/195 [==============================] - 13s 68ms/step - loss: 0.2174 - accuracy: 0.8950 - val_loss: 0.5503 - val_accuracy: 0.8192


In [6]:
model.evaluate(db_test)


195/195 [==============================] - 3s 14ms/step - loss: 0.5503 - accuracy: 0.8192


[0.550313348800708, 0.8192308]